In [1]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql import SparkSession
from pyspark.ml.classification import LinearSVC
from pyspark.ml.functions import vector_to_array
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from math import *
from pyspark.sql.window import Window


In [2]:
spark = SparkSession.builder.master("local[1]")\
          .appName("sunshine_v2")\
          .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/31 13:18:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Random Recrutement

In [16]:
df_commission = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/youssouf/Downloads/COMMISSION_RECRUTEMENT_202203230.csv") \
      .dropDuplicates()

df_commission.show()
df_commission.printSchema()

+---------+----------------+-------------+--------------------+------------------+
|DATE_JOUR|DATE_RECRUTEMENT| MOBILE_COACH|     ID_MOBILE_COACH|NOMBRE_RECRUTEMENT|
+---------+----------------+-------------+--------------------+------------------+
| 20220323|        23/03/22|     VTRV0063|            VTRV0063|                 6|
| 20220323|        23/03/22|   0701079491|PT220204.1420.940560|                 2|
| 20220323|        23/03/22|   0779734122|          0779734122|                 1|
| 20220323|        23/03/22|        SPM01|               SPM01|                 7|
| 20220323|        23/03/22|   0787792365|PT170915.1529.687886|                17|
| 20220323|        23/03/22|   0797067943|          0797067943|                 1|
| 20220323|        23/03/22|   0777142349|          0777142349|                 1|
| 20220323|        23/03/22|   0707896915|PT220126.1425.832696|                15|
| 20220323|        23/03/22|     WXZZ0108|            WXZZ0108|                 1|
| 20

In [23]:
df_commission = df_commission.withColumn("COMMISSION", col("NOMBRE_RECRUTEMENT").cast("int")/10) \
                             .withColumn("COMMISSION", col("COMMISSION").cast("int")*3500)

In [24]:
df_commission.show()

+---------+----------------+-------------+--------------------+------------------+----------+
|DATE_JOUR|DATE_RECRUTEMENT| MOBILE_COACH|     ID_MOBILE_COACH|NOMBRE_RECRUTEMENT|COMMISSION|
+---------+----------------+-------------+--------------------+------------------+----------+
| 20220323|        23/03/22|     VTRV0063|            VTRV0063|                 6|         0|
| 20220323|        23/03/22|   0701079491|PT220204.1420.940560|                 2|         0|
| 20220323|        23/03/22|   0779734122|          0779734122|                 1|         0|
| 20220323|        23/03/22|        SPM01|               SPM01|                 7|         0|
| 20220323|        23/03/22|   0787792365|PT170915.1529.687886|                17|      3500|
| 20220323|        23/03/22|   0797067943|          0797067943|                 1|         0|
| 20220323|        23/03/22|   0777142349|          0777142349|                 1|         0|
| 20220323|        23/03/22|   0707896915|PT220126.1425.8326

In [27]:
df_commission.repartition(1) \
            .write \
            .mode("overwrite") \
            .format("csv") \
            .option("header","true") \
            .option("delimiter",";") \
            .save("/Users/youssouf/Downloads/COMMISSION_RECRUTEMENT_202203230_new.csv")

### Random Activation

In [66]:
df_subscribers = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/youssouf/Downloads/FICHIER_SUBSCRIBERS_20220323.csv") \
      .dropDuplicates()

df_subscribers.show()
df_subscribers.printSchema()

+--------------------+--------------------+--------------------+------------------+----------+----------------+-----------------+--------------+---------------+----------+-------------------+-----------+--------+--------------------+------------+-------+----+-----------+--------+--------------+----------+-------------+-------+--------------------+------+---------+--------------+-------------------+--------------------+-----------------+-----------------+---------------+-------------+-------------------+--------------------+--------------------+------------+----------------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------+----------+---------------+----------+------------------------+-------+-------------------+----------+--------------------+------------+----------+------------+----------+-------------+---------+-------------------+------------------+----------------+--------------+-----+--------------+----------

In [67]:
df_subscribers_tmp = df_subscribers.select("PARENT_USER_ID","PARENT_USER_MSISDN", "MSISDN", "USER_GRADE_NAME","CREATION_DATE", "CREATED_BY", "CREATED_BY_MSISDN")
df_subscribers_tmp.show()


+------------------+----------+-------------------+--------------------+-----------------+
|PARENT_USER_MSISDN|    MSISDN|      CREATION_DATE|          CREATED_BY|CREATED_BY_MSISDN|
+------------------+----------+-------------------+--------------------+-----------------+
|        0789128522|0701598324|23/03/2022 10:47:48|PT190226.1610.344661|       0789128522|
|        0702590191|0702590191|22/03/2022 14:05:21|PT220322.1405.691746|             null|
|        0788521439|0788521439|21/03/2022 11:57:37|PT220321.1157.673833|             null|
|        0779759262|0779759262|21/03/2022 09:42:49|PT220321.0942.670547|             null|
|        0708610955|0778079078|23/03/2022 13:26:33|PT190416.1024.174113|       0708610955|
|        0701218900|0701218900|22/03/2022 13:54:47|PT220322.1354.691516|             null|
|        0788182150|0749789618|23/03/2022 12:25:15|PT180502.1248.508989|       0788182150|
|        0798418729|0704760928|23/03/2022 18:04:38|PT211021.1001.680990|       0798418729|

In [88]:
df_subscribers_tmp.filter(col("MSISDN") == "0779485556").show()

+------------------+----------+-------------------+----------+-----------------+
|PARENT_USER_MSISDN|    MSISDN|      CREATION_DATE|CREATED_BY|CREATED_BY_MSISDN|
+------------------+----------+-------------------+----------+-----------------+
|           f_gyoro|0779485556|23/03/2022 14:59:01|   f_gyoro|          f_gyoro|
+------------------+----------+-------------------+----------+-----------------+



In [68]:
df_all_users = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/youssouf/Downloads/FICHIER_ALL_USERS_20220323.csv") \
      .dropDuplicates()

df_all_users.show()
df_all_users.printSchema()

+--------------------+--------------------+--------------------+------------------+----------+----------------+--------------------+--------------------+---------------+----------+-------------------+--------------------+--------------------+-----+----------+-------------+----+-------------+--------+--------------+----------+-------------+-------+--------------------+------+--------------------+--------------+-------------------+--------------------+-----------------+-----------------+-------------------+--------------+-------------------+--------------------+--------------------+------------+----------------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+----------+------------------------+----------+-------------------+--------------------+--------------------+------------+---------+-----------+----------+-------------+--------------------+-------------------+---------

In [69]:
df_apgl = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/youssouf/Downloads/FICHIER_APGL_20220323.csv") \
      .dropDuplicates()

df_apgl.show()
df_apgl.printSchema()

+-------------+---------------+--------------------+--------------------+------------+----------------+-------------------+---------------------+------------------+------------------+-------------------+--------------------+---------------------+-------------------+-------+--------------------+---------------------+--------------------+----------------------+------------------+--------------------+------------------+--------------------+--------------------+--------------------+------------------+--------------------+----------------+--------------+----------------+---------------------+-----------------+------------+-----+----------+---------------------+------------------------+-----------------------+
|SENDER_MSISDN|RECEIVER_MSISDN|      SENDER_USER_ID|    RECEIVER_USER_ID|SERVICE_TYPE|TRANSFER_SUBTYPE|SENDER_COUNTRY_CODE|RECEIVER_COUNTRY_CODE|TRANSACTION_STATUS|SENDER_PRE_BALANCE|SENDER_POST_BALANCE|RECEIVER_PRE_BALANCE|RECEIVER_POST_BALANCE|   REFERENCE_NUMBER|REMARKS|      TRANSA

In [ ]:
#df_apgl.select("SERVICE_TYPE").distinct().show(100)

In [ ]:
#df_apgl.select("TRANSACTION_TAG").distinct().show(100)

In [ ]:
#df_apgl.select("TRANSFER_SUBTYPE").distinct().show(100)

In [70]:
df_apgl_tmp = df_apgl.select("SENDER_MSISDN", "RECEIVER_MSISDN", "SERVICE_TYPE", "TRANSACTION_DATE_TIME", "TRANSACTION_AMOUNT") \
                     .filter(col("SERVICE_TYPE").isin(["CASHOUT", "MERCHPAY"]))


In [71]:
df_apgl_tmp.show()

+-------------+---------------+------------+---------------------+------------------+
|SENDER_MSISDN|RECEIVER_MSISDN|SERVICE_TYPE|TRANSACTION_DATE_TIME|TRANSACTION_AMOUNT|
+-------------+---------------+------------+---------------------+------------------+
|   0759341269|     0779319612|    MERCHPAY|  23/03/2022 00:00:54|             200.0|
|   0749322092|     0757662915|    MERCHPAY|  23/03/2022 00:02:04|             100.0|
|   0779184857|     0768622019|    MERCHPAY|  23/03/2022 00:02:37|             100.0|
|   0758677896|     0768622019|    MERCHPAY|  23/03/2022 00:03:23|            1500.0|
|   0709557515|     0759094730|    MERCHPAY|  23/03/2022 00:03:52|             100.0|
|   0709532662|     0769790187|    MERCHPAY|  23/03/2022 00:04:29|             100.0|
|   0768443148|     0787631898|    MERCHPAY|  23/03/2022 00:06:17|             150.0|
|   0747246356|     0779692523|    MERCHPAY|  23/03/2022 00:08:59|             500.0|
|   0707871362|     0758761770|    MERCHPAY|  23/03/20

In [72]:
df_apgl_tmp = df_apgl_tmp.withColumn("DATE_TRANSACTION", substring(col("TRANSACTION_DATE_TIME"), 1, 10))

In [87]:
df_apgl_tmp.filter(col("SENDER_MSISDN") == "0779485556").show()

+-------------+---------------+------------+---------------------+------------------+----------------+
|SENDER_MSISDN|RECEIVER_MSISDN|SERVICE_TYPE|TRANSACTION_DATE_TIME|TRANSACTION_AMOUNT|DATE_TRANSACTION|
+-------------+---------------+------------+---------------------+------------------+----------------+
|   0779485556|     0748265283|    MERCHPAY|  23/03/2022 15:29:23|             150.0|      23/03/2022|
|   0779485556|     0777547725|     CASHOUT|  23/03/2022 15:17:59|           38000.0|      23/03/2022|
+-------------+---------------+------------+---------------------+------------------+----------------+



### APGL AND SUBSCRIBERS

In [82]:
df_final = df_subscribers_tmp.join(df_apgl_tmp, df_subscribers_tmp.MSISDN == df_apgl_tmp.SENDER_MSISDN, "left_outer") \
                    .dropDuplicates()
         #           .withColumn("TRANSACTION_DATE_TIME", substring(col("TRANSACTION_DATE_TIME")))

In [83]:
df_final.show()
df_final.printSchema()

+------------------+----------+-------------------+--------------------+-----------------+-------------+---------------+------------+---------------------+------------------+----------------+
|PARENT_USER_MSISDN|    MSISDN|      CREATION_DATE|          CREATED_BY|CREATED_BY_MSISDN|SENDER_MSISDN|RECEIVER_MSISDN|SERVICE_TYPE|TRANSACTION_DATE_TIME|TRANSACTION_AMOUNT|DATE_TRANSACTION|
+------------------+----------+-------------------+--------------------+-----------------+-------------+---------------+------------+---------------------+------------------+----------------+
|        0700137710|0700137710|21/03/2022 10:20:57|PT220321.1020.671345|             null|         null|           null|        null|                 null|              null|            null|
|        0700140561|0700140561|22/03/2022 16:24:29|PT220322.1624.695943|             null|         null|           null|        null|                 null|              null|            null|
|        0757688723|0700488172|23/03/202

In [84]:
df_final = df_final.select("MSISDN", "DATE_TRANSACTION", "CREATED_BY_MSISDN", "CREATED_BY", "CREATION_DATE", 
                           "SENDER_MSISDN", "TRANSACTION_AMOUNT","PARENT_USER_ID","PARENT_USER_MSISDN", 
                           "USER_GRADE_NAME")

windowMsisdnDateTransaction  = Window.partitionBy("MSISDN", "DATE_TRANSACTION")

df_final = df_final.withColumn("SUM_TRANSACTION_AMOUNT", sum("TRANSACTION_AMOUNT").over(windowMsisdnDateTransaction)) \
                    .dropDuplicates()
df_final.show()
df_final.printSchema()

+----------+----------------+-----------------+--------------------+-------------------+-------------+------------------+----------------------+
|    MSISDN|DATE_TRANSACTION|CREATED_BY_MSISDN|          CREATED_BY|      CREATION_DATE|SENDER_MSISDN|TRANSACTION_AMOUNT|SUM_TRANSACTION_AMOUNT|
+----------+----------------+-----------------+--------------------+-------------------+-------------+------------------+----------------------+
|0700137710|            null|             null|PT220321.1020.671345|21/03/2022 10:20:57|         null|              null|                  null|
|0700140561|            null|             null|PT220322.1624.695943|22/03/2022 16:24:29|         null|              null|                  null|
|0700488172|            null|       0757688723|PT210205.1147.247557|23/03/2022 17:28:10|         null|              null|                  null|
|0700488301|            null|       0757688723|PT210205.1147.247557|23/03/2022 12:00:01|         null|              null|         

In [85]:
df_final = df_final.filter( (col("SUM_TRANSACTION_AMOUNT") >= 2000) & 
                            (substring(col("CREATION_DATE"), 1, 10) == "23/03/2022")) \
                    .select("MSISDN","CREATION_DATE","CREATED_BY_MSISDN","SENDER_MSISDN",
                            "DATE_TRANSACTION","SUM_TRANSACTION_AMOUNT","PARENT_USER_ID",
                            "PARENT_USER_MSISDN", "USER_GRADE_NAME") \
                    .dropDuplicates()
df_final.show()

+----------+-------------------+-----------------+-------------+----------------+----------------------+
|    MSISDN|      CREATION_DATE|CREATED_BY_MSISDN|SENDER_MSISDN|DATE_TRANSACTION|SUM_TRANSACTION_AMOUNT|
+----------+-------------------+-----------------+-------------+----------------+----------------------+
|0703478681|23/03/2022 17:31:31|       0701291185|   0703478681|      23/03/2022|                2000.0|
|0701981432|23/03/2022 12:44:21|       0767484544|   0701981432|      23/03/2022|              150000.0|
|0702945585|23/03/2022 12:35:12|       0787791208|   0702945585|      23/03/2022|              100000.0|
|0707490095|23/03/2022 10:48:57|       0707490095|   0707490095|      23/03/2022|               65150.0|
|0768624994|23/03/2022 12:53:56|       0768624994|   0768624994|      23/03/2022|                5000.0|
|0701810849|23/03/2022 11:24:14|        f_sbamba1|   0701810849|      23/03/2022|               53000.0|
|0703474741|23/03/2022 11:09:12|       0701081407|   07

In [86]:
windowCreatedMsisdn  = Window.partitionBy("CREATED_BY_MSISDN")

df_final = df_final.withColumn("COUNT_CLIENT", count("MSISDN").over(windowCreatedMsisdn)) \
                    .withColumn("COMMISSION", col("COUNT_CLIENT")*1500)
df_final.show()
df_final.printSchema()

+----------+-------------------+-----------------+-------------+----------------+----------------------+------------+----------+
|    MSISDN|      CREATION_DATE|CREATED_BY_MSISDN|SENDER_MSISDN|DATE_TRANSACTION|SUM_TRANSACTION_AMOUNT|COUNT_CLIENT|COMMISSION|
+----------+-------------------+-----------------+-------------+----------------+----------------------+------------+----------+
|0702432222|23/03/2022 13:25:07|      mf_cangoran|   0702432222|      23/03/2022|              140000.0|           1|      1500|
|0779485556|23/03/2022 14:59:01|          f_gyoro|   0779485556|      23/03/2022|               38150.0|           1|      1500|
|0701727317|23/03/2022 11:38:52|        c_angoran|   0701727317|      23/03/2022|                2000.0|           1|      1500|
|0799134025|23/03/2022 15:54:17|         FDXQ3762|   0799134025|      23/03/2022|                5000.0|           1|      1500|
|0779992521|23/03/2022 12:21:29|    f_mouattara02|   0779992521|      23/03/2022|               1

In [89]:
df_final = df_final.select(col("CREATED_BY_MSISDN").alias("MOBILE_COACH"), col("COUNT_CLIENT").alias("NOMBRE_CLIENT"), 
                           col("COMMISSION"), col("DATE_TRANSACTION").alias("DATE_JOUR")) \
                    .dropDuplicates()

df_final.show()
df_final.printSchema()


+-------------+-------------+----------+----------+
| MOBILE_COACH|NOMBRE_CLIENT|COMMISSION| DATE_JOUR|
+-------------+-------------+----------+----------+
|  mf_cangoran|            1|      1500|23/03/2022|
|      f_gyoro|            1|      1500|23/03/2022|
|    c_angoran|            1|      1500|23/03/2022|
|     FDXQ3762|            1|      1500|23/03/2022|
|f_mouattara02|            1|      1500|23/03/2022|
|   0707490095|            1|      1500|23/03/2022|
|   0768624994|            1|      1500|23/03/2022|
|    f_sbamba1|            1|      1500|23/03/2022|
|   0767484544|            1|      1500|23/03/2022|
|      f_ayapi|            1|      1500|23/03/2022|
|   0768299687|            1|      1500|23/03/2022|
|   0787791208|            1|      1500|23/03/2022|
|   0702559282|            1|      1500|23/03/2022|
|   0701081407|            1|      1500|23/03/2022|
|     VPTQ8657|            1|      1500|23/03/2022|
|   0700025438|            1|      1500|23/03/2022|
|      f_huy

In [ ]:
df_final

In [55]:
df_final.filter(col("MOBILE_COACH") == "mf_cangoran").show()

+------------+-------------------+----------+-------------+----------+----------------+
|MOBILE_COACH|   DATE_RECRUTEMENT|    CLIENT|NOMBRE_CLIENT|COMMISSION|DATE_TRANSACTION|
+------------+-------------------+----------+-------------+----------+----------------+
| mf_cangoran|23/03/2022 13:25:07|0702432222|            4|      6000|      23/03/2022|
+------------+-------------------+----------+-------------+----------+----------------+



In [ ]:
|DATE_JOUR|DATE_RECRUTEMENT| MOBILE_COACH|     ID_MOBILE_COACH|NOMBRE_RECRUTEMENT|COMMISSION|


In [90]:
df_final.repartition(1) \
            .write \
            .mode("overwrite") \
            .format("csv") \
            .option("header","true") \
            .option("delimiter",";") \
            .save("/Users/youssouf/Downloads/COMMISSION_ACTIVATION_202203230.csv")

In [44]:
df_final = df_subscribers_tmp.join(df_apgl_tmp, df_subscribers_tmp.MSISDN == df_apgl_tmp.RECEIVER_MSISDN, "left_outer") \
                    .dropDuplicates()
         #           .withColumn("TRANSACTION_DATE_TIME", substring(col("TRANSACTION_DATE_TIME")))
    
    
windowSpec  = Window.partitionBy("MSISDN", "DATE_TRANSACTION")

df_final = df_final.withColumn("SUM_TRANSACTION_AMOUNT",sum("TRANSACTION_AMOUNT").over(windowSpec)).dropDuplicates()
df_final.show()
df_final.printSchema()


df_final = df_final.filter( (col("SUM_TRANSACTION_AMOUNT") >= 2000) & 
                            (substring(col("CREATION_DATE"), 1, 10) == "23/03/2022")) \
                    .select("PARENT_USER_MSISDN","MSISDN","CREATION_DATE","CREATED_BY_MSISDN","RECEIVER_MSISDN",
                            "SERVICE_TYPE","DATE_TRANSACTION","SUM_TRANSACTION_AMOUNT")
df_final.show()

+------------------+----------+-------------------+--------------------+-----------------+-------------+---------------+------------+---------------------+------------------+----------------+----------------------+
|PARENT_USER_MSISDN|    MSISDN|      CREATION_DATE|          CREATED_BY|CREATED_BY_MSISDN|SENDER_MSISDN|RECEIVER_MSISDN|SERVICE_TYPE|TRANSACTION_DATE_TIME|TRANSACTION_AMOUNT|DATE_TRANSACTION|SUM_TRANSACTION_AMOUNT|
+------------------+----------+-------------------+--------------------+-----------------+-------------+---------------+------------+---------------------+------------------+----------------+----------------------+
|        0700137710|0700137710|21/03/2022 10:20:57|PT220321.1020.671345|             null|         null|           null|        null|                 null|              null|            null|                  null|
|        0700140561|0700140561|22/03/2022 16:24:29|PT220322.1624.695943|             null|         null|           null|        null|       

In [4]:
df_subscribers = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/youssouf/Downloads/FICHIER_SUBSCRIBERS_20220323.csv") \
      .dropDuplicates()

df_subscribers.show()
df_subscribers.printSchema()

+--------------------+--------------------+--------------------+------------------+----------+----------------+-----------------+--------------+---------------+----------+-------------------+-----------+--------+--------------------+------------+-------+----+-----------+--------+--------------+----------+-------------+-------+--------------------+------+---------+--------------+-------------------+--------------------+-----------------+-----------------+---------------+-------------+-------------------+--------------------+--------------------+------------+----------------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------+----------+---------------+----------+------------------------+-------+-------------------+----------+--------------------+------------+----------+------------+----------+-------------+---------+-------------------+------------------+----------------+--------------+-----+--------------+----------

In [5]:
df_subscribers_tmp = df_subscribers.select("PARENT_USER_ID","PARENT_USER_MSISDN", "MSISDN", "USER_GRADE_NAME","CREATION_DATE", "CREATED_BY", "CREATED_BY_MSISDN")
df_subscribers_tmp.show()


+--------------------+------------------+----------+--------------------+-------------------+--------------------+-----------------+
|      PARENT_USER_ID|PARENT_USER_MSISDN|    MSISDN|     USER_GRADE_NAME|      CREATION_DATE|          CREATED_BY|CREATED_BY_MSISDN|
+--------------------+------------------+----------+--------------------+-------------------+--------------------+-----------------+
|PT190226.1610.344661|        0789128522|0701598324|Subscriber Lite B...|23/03/2022 10:47:48|PT190226.1610.344661|       0789128522|
|PT220322.1405.691746|        0702590191|0702590191|Subscriber Lite B...|22/03/2022 14:05:21|PT220322.1405.691746|             null|
|PT220321.1157.673833|        0788521439|0788521439|Subscriber Lite B...|21/03/2022 11:57:37|PT220321.1157.673833|             null|
|PT220321.0942.670547|        0779759262|0779759262|Subscriber Lite B...|21/03/2022 09:42:49|PT220321.0942.670547|             null|
|PT190416.1024.174113|        0708610955|0778079078|Subscriber Lite B

In [31]:
df_subscribers.filter(col("MSISDN") == "0779759262").select("MSISDN", "MODIFIED_ON", "MODIFIED_APPROVED_BY", 
                                                            "MODIFIED_APPROVED_ON", "DELETED_ON", "DEACTIVATION_BY",
                                                            "CREATION_DATE").show()

+----------+-------------------+--------------------+--------------------+----------+---------------+-------------------+
|    MSISDN|        MODIFIED_ON|MODIFIED_APPROVED_BY|MODIFIED_APPROVED_ON|DELETED_ON|DEACTIVATION_BY|      CREATION_DATE|
+----------+-------------------+--------------------+--------------------+----------+---------------+-------------------+
|0779759262|23/03/2022 10:40:53|                null|                null|      null|           null|21/03/2022 09:42:49|
+----------+-------------------+--------------------+--------------------+----------+---------------+-------------------+



In [32]:
df_subscribers_tmp.filter(col("CREATED_BY_MSISDN").startswith("event")).show()

+--------------+------------------+------+---------------+-------------+----------+-----------------+
|PARENT_USER_ID|PARENT_USER_MSISDN|MSISDN|USER_GRADE_NAME|CREATION_DATE|CREATED_BY|CREATED_BY_MSISDN|
+--------------+------------------+------+---------------+-------------+----------+-----------------+
+--------------+------------------+------+---------------+-------------+----------+-----------------+



## Recrutement

In [18]:
df_rejets = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/youssouf/Downloads/REJETS_NOMAD.csv") \
      .dropDuplicates()

df_rejets.show()
df_rejets.printSchema()

+----------------+-----------------+--------------+--------------------+--------------------+
|NUMERO DU COMPTE|STATUT DU DOSSIER|MOTIF DU REJET|      NOM DE L'AGENT|DATE DE SOUSCRIPTION|
+----------------+-----------------+--------------+--------------------+--------------------+
|      0757900349|           VALIDE|          null|ACTION T122 CAN 2...|    23/03/2022 17:22|
|      0788309043|           VALIDE|          null|ACTION PMO T1 202...|    23/03/2022 15:35|
|      0779350752|           VALIDE|          null|REGHI PMOT1 - pmo...|    23/03/2022 12:04|
|      0778904975|           VALIDE|          null|PDVL AMANDA PREST...|    23/03/2022 10:58|
|      0701009493|           VALIDE|          null|SEKONGO TINNILO S...|    24/03/2022 18:31|
|      0701718242|           VALIDE|          null|SEDION  KASSOUM -...|    24/03/2022 17:36|
|      0788725411|           VALIDE|          null|SORO   SEYDOU - p...|    24/03/2022 17:25|
|      0747050278|           VALIDE|          null|PCU Selle

In [19]:
df_rejets.filter(col("STATUT DU DOSSIER") == "INVALIDE").show()

+----------------+-----------------+-----------------+--------------------+--------------------+
|NUMERO DU COMPTE|STATUT DU DOSSIER|   MOTIF DU REJET|      NOM DE L'AGENT|DATE DE SOUSCRIPTION|
+----------------+-----------------+-----------------+--------------------+--------------------+
|      0768963063|         INVALIDE|DOUBLON COMPTE OM|ACTION 2AS  EVENT...|    23/03/2022 16:08|
|      0778525949|         INVALIDE|    PAS IDENTIFIE|SORO   SEYDOU - p...|    23/03/2022 17:33|
|      0769601307|         INVALIDE|    PAS IDENTIFIE|ETTIEN  HUBERSON ...|    23/03/2022 13:27|
|      0787833631|         INVALIDE|    PAS IDENTIFIE|ACTION T122 CAN I...|    23/03/2022 19:39|
|      0703478680|         INVALIDE|    PAS IDENTIFIE|ACTION 2AS  RECRU...|    23/03/2022 17:22|
|      0708127898|         INVALIDE|    PAS IDENTIFIE|PCU Seller PCU Se...|    23/03/2022 15:04|
|      0749361678|         INVALIDE|DOUBLON COMPTE OM|Kouassi Affoue Es...|    24/03/2022 17:09|
|      0777286630|         INV

In [21]:
    df_rejets = df_rejets.withColumn("DATE", substring(col("DATE DE SOUSCRIPTION"), 1, 10))

In [22]:
df_rejets.select("DATE").distinct().show(100)c

+----------+
|      DATE|
+----------+
|27/03/2022|
|25/03/2022|
|26/03/2022|
|23/03/2022|
|28/03/2022|
|24/03/2022|
+----------+



In [ ]:
   var df_subscribers = spark.read
        .option("delimiter", ";")
        .option("header", "true")
        .csv(s"/home/random/subscribers/FICHIER_SUBSCRIBERS_20220401.csv")
        .dropDuplicates()

In [2]:
import pandas as pd

In [4]:
df = pd.read_excel("/Users/youssouf/Downloads/MAJ logins ISS T2 190422.xlsx", header = 1)

In [5]:
df.head()

,CABINET,NOM ET PRENOM MC,NUMERO NOMAD,LOGIN,NUMERO OM,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,ISS,KOUASSI ELISE,07 01 56 67 98,covert222_iss_s2i_003,07 02 41 64 08,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ISS,KOUASSI JEAN CLAUDE,07 08 70 77 08,covert222_iss_s2i_005,07 00 27 71 07,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ISS,BAILLY ELIE NEVILLE,07 08 71 67 17,covert222_iss_s2i_006,07 67 47 65 59,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ISS,TIE LOU BENE ANGE CYNTHIA,07 08 92 19 22,covert222_iss_s2i_007,07 09 51 19 66,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ISS,KOUASSI ABOKAN,07 08 83 92 92,covert222_iss_s2i_009,07 57 76 95 96,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.rename(columns = {'NUMERO NOMAD':'NUMERO', 'NUMERO OM':'MOBILE_OM'}, inplace = True)

In [7]:
df.to_csv("/Users/youssouf/Downloads/mobile_coach_reel.csv", index = False)

In [8]:
pd.read_csv("/Users/youssouf/Downloads/mobile_coach_reel.csv").head()

,CABINET,NOM ET PRENOM MC,NUMERO,LOGIN,MOBILE_OM,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,ISS,KOUASSI ELISE,07 01 56 67 98,covert222_iss_s2i_003,07 02 41 64 08,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ISS,KOUASSI JEAN CLAUDE,07 08 70 77 08,covert222_iss_s2i_005,07 00 27 71 07,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ISS,BAILLY ELIE NEVILLE,07 08 71 67 17,covert222_iss_s2i_006,07 67 47 65 59,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ISS,TIE LOU BENE ANGE CYNTHIA,07 08 92 19 22,covert222_iss_s2i_007,07 09 51 19 66,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ISS,KOUASSI ABOKAN,07 08 83 92 92,covert222_iss_s2i_009,07 57 76 95 96,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_rejets = pd.read_excel("/Users/youssouf/Downloads/Rejets nomad S15_20220415.xlsx", header = 3)
df_rejets.head()

,N° ORDRE,N° DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION
0,1,60945544,701939119,VALIDE,SERY JEAN EUDES PAULIN - ext_pdv_sii_serj_5305,05/04/2022 20:19
1,2,60945515,701965112,VALIDE,JOSUE AGOUA - indep_agoua_8298,05/04/2022 10:16
2,3,60945478,789823090,VALIDE,EPONOU St�phanie Kouadio - pmot122-lean-07,05/04/2022 20:10
3,4,60945067,797286466,VALIDE,EPONOU St�phanie Kouadio - pmot122-lean-07,05/04/2022 19:28
4,5,60944898,768159126,VALIDE,MC ACTION PMO T1-2022 - tkgspmot122-06,05/04/2022 19:13


In [23]:
df_rejets.to_csv("/Users/youssouf/Downloads/rejets_nomad_S15_20220415.csv", index = False, sep = ";")

In [24]:
pd.read_csv("/Users/youssouf/Downloads/rejets_nomad_S15_20220415.csv", sep = ";").head()

,N° ORDRE,N° DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION
0,1,60945544,701939119,VALIDE,SERY JEAN EUDES PAULIN - ext_pdv_sii_serj_5305,05/04/2022 20:19
1,2,60945515,701965112,VALIDE,JOSUE AGOUA - indep_agoua_8298,05/04/2022 10:16
2,3,60945478,789823090,VALIDE,EPONOU St�phanie Kouadio - pmot122-lean-07,05/04/2022 20:10
3,4,60945067,797286466,VALIDE,EPONOU St�phanie Kouadio - pmot122-lean-07,05/04/2022 19:28
4,5,60944898,768159126,VALIDE,MC ACTION PMO T1-2022 - tkgspmot122-06,05/04/2022 19:13


In [25]:
dfs = pd.read_excel("/Users/youssouf/Downloads/MISE A JOUR NOMAD T2.xlsx", header = 0)
dfs.head()

,CABINET,NOM & PRENOMS DU MC,N° NOMAD,LOGIN,N° MOBILE OM DU MC
0,BOP,ALLANGBA BAUDELAIRE KONAN,07 00 27 99 74,godogodo_moz_t222_bop01,07 88 06 10 06
1,NaN,KOUAKOU CONSTANT,07 00 27 99 81,godogodo_moz_t222_bop02,07 09 14 43 38
2,NaN,DASSO ARMEL SCHAMA,07 00 27 99 82,godogodo_moz_t222_bop03,07 59 74 59 27
3,NaN,KARAMOKO AMINATA,07 00 27 99 85,godogodo_moz_t222_bop04,07 69 11 97 96
4,NaN,TIEKORA DIEUDONNE,07 00 27 99 86,godogodo_moz_t222_bop05,07 49 87 58 98


In [16]:
dfs.shape

(94, 5)

In [26]:
dfs.rename(columns = {'N° NOMAD':'NOMAD', 'N° MOBILE OM DU MC':'MOBILE_OM'}, inplace = True)

In [27]:
dfs.to_csv("/Users/youssouf/Downloads/mobile_coach_reel2.csv", index = False, sep =";")

In [17]:
df.shape

(77, 21)

In [3]:
df = pd.read_csv("/Users/youssouf/Downloads/Rejets_nomad_20220415.csv", sep = ";")
df.head()

,N� ORDRE,N� DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION
0,1,60945544,701939119,VALIDE,SERY JEAN EUDES PAULIN - ext_pdv_sii_serj_5305,05/04/2022 20:19
1,2,60945515,701965112,VALIDE,JOSUE AGOUA - indep_agoua_8298,05/04/2022 10:16
2,3,60945478,789823090,VALIDE,EPONOU St?phanie Kouadio - pmot122-lean-07,05/04/2022 20:10
3,4,60945067,797286466,VALIDE,EPONOU St?phanie Kouadio - pmot122-lean-07,05/04/2022 19:28
4,5,60944898,768159126,VALIDE,MC ACTION PMO T1-2022 - tkgspmot122-06,05/04/2022 19:13


In [4]:
df1 = pd.read_csv("/Users/youssouf/Downloads/Rejets_nomad_20220422.csv", sep = ";")
df1.head()

,N� ORDRE,N� DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION
0,1,61188095,797363180,VALIDE,ACTION PMO T2 2022 - djoum2022_tid53,12/04/2022 19:16
1,2,61188092,797363177,VALIDE,ACTION PMO T2 2022 - djoum2022_tid53,12/04/2022 19:14
2,3,61188090,797363176,VALIDE,ACTION PMO T2 2022 - djoum2022_tid53,12/04/2022 19:13
3,4,61185634,799032570,VALIDE,SORO SEYDOU - pmot122_districentre_23,12/04/2022 21:55
4,5,61185592,798758044,VALIDE,SORO SEYDOU - pmot122_districentre_23,12/04/2022 21:46


In [5]:
df2 = pd.read_csv("/Users/youssouf/Downloads/Rejets_nomad_20220426.csv", sep = ";")
df2.head()

,N� ORDRE,N� DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION
0,1,61404857,702569183,VALIDE,MAGIC B ACTION PMO - pmot222_magicb_3,19/04/2022 11:10
1,2,61391055,767518045,VALIDE,DIALLO IDRISS YACINE - pmot222-sii-135,19/04/2022 19:45
2,3,61385484,767950975,VALIDE,SORO SEYDOU - hpmot121_districentre_12,19/04/2022 16:37
3,4,61391166,779218319,VALIDE,ACTION PMO T2-22 - pmot222-lacoq-54,19/04/2022 19:59
4,5,61391136,789022301,VALIDE,DIALLO IDRISS YACINE - pmot222-sii-27,19/04/2022 19:57


In [6]:
df3 = pd.read_csv("/Users/youssouf/Downloads/Rejets_nomad_20220429.csv", sep = ";")
df3.head()

,N� ORDRE,N� DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION
0,1,61527136,701582403,VALIDE,ACTION PMO T2 2022 - djoum2022_tid05,22/04/2022 17:58
1,2,61527134,701582103,VALIDE,ACTION PMO T2 2022 - djoum2022_tid05,22/04/2022 17:56
2,3,61525636,767527817,VALIDE,SUITE COM SUITE COM - pmot222-suitecom-17,22/04/2022 19:59
3,4,61525629,700866327,VALIDE,ISS ACTION COVER S2I - cover_iss_s2i_060,22/04/2022 19:58
4,5,61525615,703152423,VALIDE,ACTION T122 CAN ISS - canstreet_s2i_iss_017,22/04/2022 19:56


In [7]:
df4 = pd.read_csv("/Users/youssouf/Downloads/Rejets_nomad_20220506.csv", sep = ";")
df4.head()

,N� ORDRE,N� DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION
0,1,61608051,798609443,VALIDE,PMO T4 2021 ACTION - pmot421_districentre_43,25/04/2022 12:14
1,2,61652593,701398738,VALIDE,Yao Denise - pmot222-ajs-50,25/04/2022 14:17
2,3,61638209,778968479,VALIDE,IGP ACTION GODO GODO - godogodo_igp_reghi010,25/04/2022 19:51
3,4,61638194,798067011,VALIDE,SUITE COM SUITE COM - pmot222-suitecom-24,25/04/2022 19:58
4,5,61638145,778267732,VALIDE,ISS ACTION GODO GODO - godogodo_iss_tid009,25/04/2022 19:55


In [8]:
df_f = pd.concat([df, df1, df2, df3, df4])

In [13]:
df_f["DATE_SOUSCRIPTION"] = df_f["DATE DE SOUSCRIPTION"].str.slice(0, 10)

In [14]:
df_f["DATE_SOUSCRIPTION"].unique()

array(['05/04/2022', '06/04/2022', '07/04/2022', '08/04/2022',
       '09/04/2022', '10/04/2022', '11/04/2022', '12/04/2022',
       '13/04/2022', '14/04/2022', '15/04/2022', '16/04/2022',
       '17/04/2022', '18/04/2022', '19/04/2022', '20/04/2022',
       '21/04/2022', '22/04/2022', '23/04/2022', '24/04/2022',
       '25/04/2022', '26/04/2022', '27/04/2022', '28/04/2022',
       '29/04/2022', '30/04/2022', '01/05/2022', '02/05/2022'],
      dtype=object)

In [11]:
df_f2 = df_f.drop(["DATE_SOUSCRIPTION"], axis = 1)

In [12]:
df_f2.to_csv("/Users/youssouf/Downloads/rejet_nomad_activation_202200506.csv", index = False, sep = ";")

In [17]:
df = spark.read.csv("/Users/youssouf/Downloads/rejet_nomad_activation_202200506.csv").option("delimiter", ";").option("header", True)

df.show()


AttributeError: 'DataFrame' object has no attribute 'option'

In [37]:
df = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/youssouf/Downloads/rejet_nomad_activation_202200506.csv") \
      .dropDuplicates()

df.show()

+--------+----------+---------+-----------------+--------------------+--------------------+
|N� ORDRE|N� DOSSIER|   NUMERO|STATUT DU DOSSIER|      NOM DE L'AGENT|DATE DE SOUSCRIPTION|
+--------+----------+---------+-----------------+--------------------+--------------------+
|      54|  60940293|767882147|           VALIDE|DIALLO IDRISS YAC...|    05/04/2022 16:13|
|      61|  60939830|759124873|           VALIDE|PDVL AMANDA PREST...|    05/04/2022 16:02|
|     654|  60969056|798704470|           VALIDE|PDVL AMANDA PREST...|    06/04/2022 15:18|
|     731|  60965652|788325166|           VALIDE|PDVL AMANDA PREST...|    06/04/2022 14:02|
|     751|  60964562|748372666|           VALIDE|PDVL AMANDA PREST...|    06/04/2022 13:41|
|     939|  60956771|767391266|           VALIDE|ACTION T122 CAN B...|    06/04/2022 11:24|
|    1135|  60950039|701260098|           VALIDE|PMO T2 2022 ACTIO...|    06/04/2022 09:24|
|    1139|  60949959|700314011|           VALIDE|DIALLO FATOGOMA -...|    06/04/

In [40]:
df = df.withColumn("dt", to_date(substring(col("DATE DE SOUSCRIPTION"), 1, 10), "dd/MM/yyyy"))

In [35]:
#df = df .withColumn("dt", col("dt").cast( DateType() ))

In [41]:
df.show()

+--------+----------+---------+-----------------+--------------------+--------------------+----------+
|N� ORDRE|N� DOSSIER|   NUMERO|STATUT DU DOSSIER|      NOM DE L'AGENT|DATE DE SOUSCRIPTION|        dt|
+--------+----------+---------+-----------------+--------------------+--------------------+----------+
|      54|  60940293|767882147|           VALIDE|DIALLO IDRISS YAC...|    05/04/2022 16:13|2022-04-05|
|      61|  60939830|759124873|           VALIDE|PDVL AMANDA PREST...|    05/04/2022 16:02|2022-04-05|
|     654|  60969056|798704470|           VALIDE|PDVL AMANDA PREST...|    06/04/2022 15:18|2022-04-06|
|     731|  60965652|788325166|           VALIDE|PDVL AMANDA PREST...|    06/04/2022 14:02|2022-04-06|
|     751|  60964562|748372666|           VALIDE|PDVL AMANDA PREST...|    06/04/2022 13:41|2022-04-06|
|     939|  60956771|767391266|           VALIDE|ACTION T122 CAN B...|    06/04/2022 11:24|2022-04-06|
|    1135|  60950039|701260098|           VALIDE|PMO T2 2022 ACTIO...|   

In [50]:
df.select(min(col("dt"))).first().asInstanceOf[Double]

Row(min(dt)=datetime.date(2022, 4, 5))

In [53]:
df.select(max(col("dt"))).first().get(0).asInstanceOf[String]

AttributeError: get

In [25]:
df.select("dt").distinct().show(30)

+----------+
|        dt|
+----------+
|30/04/2022|
|26/04/2022|
|25/04/2022|
|19/04/2022|
|09/04/2022|
|02/05/2022|
|15/04/2022|
|20/04/2022|
|13/04/2022|
|23/04/2022|
|05/04/2022|
|29/04/2022|
|06/04/2022|
|12/04/2022|
|18/04/2022|
|01/05/2022|
|28/04/2022|
|21/04/2022|
|11/04/2022|
|17/04/2022|
|08/04/2022|
|16/04/2022|
|22/04/2022|
|27/04/2022|
|10/04/2022|
|24/04/2022|
|14/04/2022|
|07/04/2022|
+----------+



In [14]:
df_f = pd.concat([df, df1, df2, df3])

In [15]:
df_f.head()

,N� ORDRE,N� DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION
0,1,60945544,701939119,VALIDE,SERY JEAN EUDES PAULIN - ext_pdv_sii_serj_5305,05/04/2022 20:19
1,2,60945515,701965112,VALIDE,JOSUE AGOUA - indep_agoua_8298,05/04/2022 10:16
2,3,60945478,789823090,VALIDE,EPONOU St?phanie Kouadio - pmot122-lean-07,05/04/2022 20:10
3,4,60945067,797286466,VALIDE,EPONOU St?phanie Kouadio - pmot122-lean-07,05/04/2022 19:28
4,5,60944898,768159126,VALIDE,MC ACTION PMO T1-2022 - tkgspmot122-06,05/04/2022 19:13


In [16]:
df_f["DATE_SOUSCRIPTION"] = df_f["DATE DE SOUSCRIPTION"].str.slice(0, 9)
df_f.head()

,N� ORDRE,N� DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION,DATE_SOUSCRIPTION
0,1,60945544,701939119,VALIDE,SERY JEAN EUDES PAULIN - ext_pdv_sii_serj_5305,05/04/2022 20:19,05/04/202
1,2,60945515,701965112,VALIDE,JOSUE AGOUA - indep_agoua_8298,05/04/2022 10:16,05/04/202
2,3,60945478,789823090,VALIDE,EPONOU St?phanie Kouadio - pmot122-lean-07,05/04/2022 20:10,05/04/202
3,4,60945067,797286466,VALIDE,EPONOU St?phanie Kouadio - pmot122-lean-07,05/04/2022 19:28,05/04/202
4,5,60944898,768159126,VALIDE,MC ACTION PMO T1-2022 - tkgspmot122-06,05/04/2022 19:13,05/04/202


In [17]:
df_f["DATE_SOUSCRIPTION"].unique()

array(['05/04/202', '06/04/202', '07/04/202', '08/04/202', '09/04/202',
       '10/04/202', '11/04/202', '12/04/202', '13/04/202', '14/04/202',
       '15/04/202', '16/04/202', '17/04/202', '18/04/202', '19/04/202',
       '20/04/202', '21/04/202', '22/04/202', '23/04/202', '24/04/202'],
      dtype=object)

In [ ]:
df_f["DATE_SOUSCRIPTION"]

In [19]:
df_f1 = df_f[~df_f["DATE_SOUSCRIPTION"].isin(['20/04/202', '21/04/202', '22/04/202', '23/04/202', '24/04/202'])]

In [21]:
df_f2 = df_f1.drop(["DATE_SOUSCRIPTION"], axis = 1)
df_f2.head()

,N� ORDRE,N� DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION
0,1,60945544,701939119,VALIDE,SERY JEAN EUDES PAULIN - ext_pdv_sii_serj_5305,05/04/2022 20:19
1,2,60945515,701965112,VALIDE,JOSUE AGOUA - indep_agoua_8298,05/04/2022 10:16
2,3,60945478,789823090,VALIDE,EPONOU St?phanie Kouadio - pmot122-lean-07,05/04/2022 20:10
3,4,60945067,797286466,VALIDE,EPONOU St?phanie Kouadio - pmot122-lean-07,05/04/2022 19:28
4,5,60944898,768159126,VALIDE,MC ACTION PMO T1-2022 - tkgspmot122-06,05/04/2022 19:13


In [23]:
df_f2.to_csv("/Users/youssouf/Downloads/prime_transport_20220419.csv", index = False, sep = ";")

In [6]:
df["DATE_SOUSCRIPTION"] = df["DATE DE SOUSCRIPTION"].str.slice(0, 9)
df.head()

,N� ORDRE,N� DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION,DATE_SOUSCRIPTION
0,1,60945544,701939119,VALIDE,SERY JEAN EUDES PAULIN - ext_pdv_sii_serj_5305,05/04/2022 20:19,05/04/202
1,2,60945515,701965112,VALIDE,JOSUE AGOUA - indep_agoua_8298,05/04/2022 10:16,05/04/202
2,3,60945478,789823090,VALIDE,EPONOU St?phanie Kouadio - pmot122-lean-07,05/04/2022 20:10,05/04/202
3,4,60945067,797286466,VALIDE,EPONOU St?phanie Kouadio - pmot122-lean-07,05/04/2022 19:28,05/04/202
4,5,60944898,768159126,VALIDE,MC ACTION PMO T1-2022 - tkgspmot122-06,05/04/2022 19:13,05/04/202


In [11]:
df = pd.read_csv("/Users/diakite/Downloads/Telegram/Rejets_nomad_20220415.csv", sep = ";", encoding= 'unicode_escape')
df.head()

df1 = pd.read_csv("/Users/diakite/Downloads/Telegram/Rejets_nomad_20220422.csv", sep = ";", encoding= 'unicode_escape')
df1.head()

df2 = pd.read_csv("/Users/diakite/Downloads/Telegram/Rejets_nomad_20220426.csv", sep = ";", encoding= 'unicode_escape')
df2.head()

df3 = pd.read_csv("/Users/diakite/Downloads/Telegram/Rejets_nomad_20220429.csv", sep = ";", encoding= 'unicode_escape')
df3.head()

df4 = pd.read_csv("/Users/diakite/Downloads/Telegram/Rejets_nomad_20220506.csv", sep = ";", encoding= 'unicode_escape')
df4.head()


,N° ORDRE,N° DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION
0,1,61608051,798609443,VALIDE,PMO T4 2021 ACTION - pmot421_districentre_43,25/04/2022 12:14
1,2,61652593,701398738,VALIDE,Yao Denise - pmot222-ajs-50,25/04/2022 14:17
2,3,61638209,778968479,VALIDE,IGP ACTION GODO GODO - godogodo_igp_reghi010,25/04/2022 19:51
3,4,61638194,798067011,VALIDE,SUITE COM SUITE COM - pmot222-suitecom-24,25/04/2022 19:58
4,5,61638145,778267732,VALIDE,ISS ACTION GODO GODO - godogodo_iss_tid009,25/04/2022 19:55


In [12]:
df_f = pd.concat([df, df1, df2, df3, df4])

In [13]:
df_f["DATE_SOUSCRIPTION"] = df_f["DATE DE SOUSCRIPTION"].str.slice(0, 10)

df_f["DATE_SOUSCRIPTION"].unique()

array(['05/04/2022', '06/04/2022', '07/04/2022', '08/04/2022',
       '09/04/2022', '10/04/2022', '11/04/2022', '12/04/2022',
       '13/04/2022', '14/04/2022', '15/04/2022', '16/04/2022',
       '17/04/2022', '18/04/2022', '19/04/2022', '20/04/2022',
       '21/04/2022', '22/04/2022', '23/04/2022', '24/04/2022',
       '25/04/2022', '26/04/2022', '27/04/2022', '28/04/2022',
       '29/04/2022', '30/04/2022', '01/05/2022', '02/05/2022'],
      dtype=object)

In [14]:
df_f1 = df_f[~df_f["DATE_SOUSCRIPTION"].isin(['05/04/2022', '25/04/2022', '26/04/2022', '27/04/2022', '28/04/2022',
       '29/04/2022', '30/04/2022', '01/05/2022', '02/05/2022'])]

In [15]:
df_f1["DATE_SOUSCRIPTION"].unique()

array(['06/04/2022', '07/04/2022', '08/04/2022', '09/04/2022',
       '10/04/2022', '11/04/2022', '12/04/2022', '13/04/2022',
       '14/04/2022', '15/04/2022', '16/04/2022', '17/04/2022',
       '18/04/2022', '19/04/2022', '20/04/2022', '21/04/2022',
       '22/04/2022', '23/04/2022', '24/04/2022'], dtype=object)

In [16]:
df_f2 = df_f1.drop(["DATE_SOUSCRIPTION"], axis = 1)

In [17]:
df_f2.to_csv("/Users/diakite/Downloads/rejet_nomad_activation_BOP_20220424.csv", index = False, sep = ";")

In [18]:
df_f3 = df_f[~df_f["DATE_SOUSCRIPTION"].isin(['05/04/2022', '06/04/2022', '07/04/2022', '08/04/2022',
       '09/04/2022', '10/04/2022', '25/04/2022', '26/04/2022', '27/04/2022', '28/04/2022',
       '29/04/2022', '30/04/2022', '01/05/2022', '02/05/2022'])]

In [19]:
df_f3.head()

,N° ORDRE,N° DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION,DATE_SOUSCRIPTION
3687,3688,61157728,777466897,VALIDE,BOP ACTION GODO GODO - godogodo_bop_mos015,11/04/2022 16:54,11/04/2022
3688,3689,61127582,768812098,VALIDE,PMO T4 2021 ACTION - pmot421_districentre_43,11/04/2022 14:51,11/04/2022
3689,3690,61135334,789960915,VALIDE,MAGIC B ACTION PMO - pmot222_magicb_25,11/04/2022 16:52,11/04/2022
3690,3691,61147733,799077300,VALIDE,IGP ACTION GODO GODO - godogodo_igp_sc014,11/04/2022 17:17,11/04/2022
3691,3692,61140875,799566607,VALIDE,ACTION PMO T2 2022 - djoum2022_tid24,11/04/2022 21:49,11/04/2022


In [20]:
df_f3 = df_f3.drop(["DATE_SOUSCRIPTION"], axis = 1)

In [21]:
df_f3.to_csv("/Users/diakite/Downloads/rejet_nomad_activation_ISS_20220424.csv", index = False, sep = ";")

In [22]:
df_f4 = df_f[df_f["DATE_SOUSCRIPTION"].isin(['19/04/2022', '20/04/2022',
       '21/04/2022', '22/04/2022', '23/04/2022', '24/04/2022',
       '25/04/2022', '26/04/2022', '27/04/2022', '28/04/2022',
       '29/04/2022', '30/04/2022'])]

In [ ]:
df_f4 = df_f4.drop(["DATE_SOUSCRIPTION"], axis = 1)

In [23]:
df_f4.to_csv("/Users/diakite/Downloads/rejet_nomad_20220430.csv", index = False, sep = ";")

In [ ]:
df_f2 = df_f.drop(["DATE_SOUSCRIPTION"], axis = 1)

#df_f2.to_csv("/Users/youssouf/Downloads/rejet_nomad_activation_202200506.csv", index = False, sep = ";")

### Comparison MC et Nomad 

In [24]:
df4 = pd.read_csv("/Users/diakite/Downloads/Telegram/Rejets_nomad_20220506.csv", sep = ";", encoding= 'unicode_escape')
df4.head()

,N° ORDRE,N° DOSSIER,NUMERO,STATUT DU DOSSIER,NOM DE L'AGENT,DATE DE SOUSCRIPTION
0,1,61608051,798609443,VALIDE,PMO T4 2021 ACTION - pmot421_districentre_43,25/04/2022 12:14
1,2,61652593,701398738,VALIDE,Yao Denise - pmot222-ajs-50,25/04/2022 14:17
2,3,61638209,778968479,VALIDE,IGP ACTION GODO GODO - godogodo_igp_reghi010,25/04/2022 19:51
3,4,61638194,798067011,VALIDE,SUITE COM SUITE COM - pmot222-suitecom-24,25/04/2022 19:58
4,5,61638145,778267732,VALIDE,ISS ACTION GODO GODO - godogodo_iss_tid009,25/04/2022 19:55


In [26]:
from spylon_kernel import register_ipython_magics
register_ipython_magics()

In [27]:
%%scala

val x = 3

Intitializing Scala interpreter ...

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=sunshine_v2, master=local[1]) created by getOrCreate at /var/folders/b7/pj4f3ws14fl09641x7pq364w0000gn/T/ipykernel_93452/1297210238.py:1 

In [28]:
df = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/diakite/Downloads/Telegram/Rejets_nomad_20220506.csv") \
      .dropDuplicates()

df.show()
df.printSchema()

+--------+----------+----------+-----------------+--------------------+--------------------+
|N� ORDRE|N� DOSSIER|    NUMERO|STATUT DU DOSSIER|      NOM DE L'AGENT|DATE DE SOUSCRIPTION|
+--------+----------+----------+-----------------+--------------------+--------------------+
|     163|  61634372|0700422178|           VALIDE|ACTION PMO T2 202...|    25/04/2022 17:47|
|     183|  61634078|0779234793|           VALIDE|IGP ACTION GODO G...|    25/04/2022 17:36|
|     189|  61633921|0768934557|           VALIDE|ACTION PMO T2 202...|    25/04/2022 17:38|
|     484|  61626630|0747497836|           VALIDE|ACTION PMO T2-22 ...|    25/04/2022 15:55|
|     625|  61623963|0797865752|           VALIDE|PDVL AMANDA PREST...|    25/04/2022 15:22|
|    1829|  61597697|0797289124|           VALIDE|ISS ACTION GODO G...|    25/04/2022 10:20|
|    1859|  61596960|0777260693|           VALIDE|ISS ACTION GODO G...|    25/04/2022 10:12|
|    1902|  61596149|0789294232|           VALIDE|2AS ACTION GODO G...

In [36]:
df1 = df.withColumn("DATE", substring(col("DATE DE SOUSCRIPTION"), 1, 10))  \
       .filter(col("STATUT DU DOSSIER") != "INVALIDE") 

In [37]:
df1.show()

+--------+----------+----------+-----------------+--------------------+--------------------+----------+
|N� ORDRE|N� DOSSIER|    NUMERO|STATUT DU DOSSIER|      NOM DE L'AGENT|DATE DE SOUSCRIPTION|      DATE|
+--------+----------+----------+-----------------+--------------------+--------------------+----------+
|     163|  61634372|0700422178|           VALIDE|ACTION PMO T2 202...|    25/04/2022 17:47|25/04/2022|
|     183|  61634078|0779234793|           VALIDE|IGP ACTION GODO G...|    25/04/2022 17:36|25/04/2022|
|     189|  61633921|0768934557|           VALIDE|ACTION PMO T2 202...|    25/04/2022 17:38|25/04/2022|
|     484|  61626630|0747497836|           VALIDE|ACTION PMO T2-22 ...|    25/04/2022 15:55|25/04/2022|
|     625|  61623963|0797865752|           VALIDE|PDVL AMANDA PREST...|    25/04/2022 15:22|25/04/2022|
|    1829|  61597697|0797289124|           VALIDE|ISS ACTION GODO G...|    25/04/2022 10:20|25/04/2022|
|    1859|  61596960|0777260693|           VALIDE|ISS ACTION GOD

In [41]:
df1.filter(col("DATE") == "28/04/2022").show()

+--------+----------+----------+-----------------+--------------------+--------------------+----------+
|N� ORDRE|N� DOSSIER|    NUMERO|STATUT DU DOSSIER|      NOM DE L'AGENT|DATE DE SOUSCRIPTION|      DATE|
+--------+----------+----------+-----------------+--------------------+--------------------+----------+
|    4936|  61752305|0787987535|           VALIDE|2AS ACTION GODO G...|    28/04/2022 14:35|28/04/2022|
|    5019|  61749820|0702992199|           VALIDE|IGP ACTION RESESR...|    28/04/2022 13:50|28/04/2022|
|    5038|  61749277|0702623215|           VALIDE|ISS ACTION COVER ...|    28/04/2022 13:45|28/04/2022|
|    5300|  61741045|0778937638|           VALIDE|ACTION PMO T2 202...|    28/04/2022 11:37|28/04/2022|
|   13860|  61763773|0769253228|      VALIDE LITE|ACTION PMO T2 202...|    28/04/2022 19:42|28/04/2022|
|   13942|  61761603|0704345704|      VALIDE LITE|ACTION PMO T2 202...|    28/04/2022 17:59|28/04/2022|
|   13982|  61760461|0758058096|      VALIDE LITE|PCU Seller PCU

In [ ]:
df = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/diakite/Downloads/Telegram/Rejets_nomad_20220506.csv") \
      .dropDuplicates()

In [51]:
def check_number(date) :
    
    df = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/diakite/Downloads/Telegram/Rejets_nomad_20220506.csv") \
      .withColumn("DATE", substring(col("DATE DE SOUSCRIPTION"), 1, 10))  \
      .filter( (col("STATUT DU DOSSIER") != "INVALIDE") & (col("DATE") == date))  \
      .dropDuplicates()
      
    return df.show(truncate=False)
    

In [52]:
check_number("28/04/2022")

+--------+----------+----------+-----------------+------------------------------------------------------+--------------------+----------+
|N� ORDRE|N� DOSSIER|NUMERO    |STATUT DU DOSSIER|NOM DE L'AGENT                                        |DATE DE SOUSCRIPTION|DATE      |
+--------+----------+----------+-----------------+------------------------------------------------------+--------------------+----------+
|4724    |61757801  |0700489017|VALIDE           |ISS ACTION GODO GODO - godogodo_iss_tid007            |28/04/2022 16:41    |28/04/2022|
|5376    |61738613  |0701940972|VALIDE           |2AS ACTION GODO GODO - godogodo_2as_tkgs013           |28/04/2022 11:04    |28/04/2022|
|5386    |61738266  |0778885809|VALIDE           |ACTION PMO T2 2022 - djoum2022_tid78                  |28/04/2022 10:59    |28/04/2022|
|5394    |61738127  |0767729498|VALIDE           |2AS ACTION GODO GODO - godogodo_2as_prim018           |28/04/2022 10:57    |28/04/2022|
|13931   |61761887  |0704345275|VA

In [3]:
df = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/diakite/Downloads/Telegram Desktop/random_drdi_activation_iss_%2A.csv") \
      .dropDuplicates()

In [4]:
df.count()

10

In [5]:
df.show()

+----------+------------+---------------+----------+----------------+--------------------+------------+----------+-------------+--------------------+
|COMMISSION|COUNT_CLIENT|DATE_CHARGEMENT| DATE_JOUR|DATE_TRANSACTION|               LOGIN|MOBILE_COACH| MOBILE_OM|NOMBRE_CLIENT|   NOM_PRENOMS_DU_MC|
+----------+------------+---------------+----------+----------------+--------------------+------------+----------+-------------+--------------------+
|    1500.0|        null|     2022-05-27|2022-04-24|            null| godogodo_iss_idt011|  0700274220|0747799758|            1|  KOUADIO YAO PRIVAT|
|    1500.0|        null|     2022-05-27|2022-05-11|            null|godogodo_iss_dist005|  0701566810|0708210020|            1|      OUATTARA SIAKA|
|    1500.0|        null|     2022-05-27|2022-04-27|            null|godogodo_iss_aman001|  0701566835|0709182047|            1|   OUATTARA SALY SIE|
|    1500.0|        null|     2022-05-27|2022-04-21|            null|   cover_iss_s2i_035|  07015667

In [10]:
def f(path, flag) :
    
    df = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/diakite/Downloads/Telegram Desktop/" + path) \
      .dropDuplicates()
    
    
    df.show()
    
    df = df.drop("COUNT_CLIENT")
    df = df.drop("DATE_TRANSACTION")
    
    df.toPandas().to_csv("/Users/diakite/Downloads/Telegram Desktop/" + flag + path , index = False, sep = ";")
    
    

In [12]:
f("random_drdi_activation_iss_%2A.csv",  "iss")

f("random_drdi_activation_igp_%2A.csv",  "igp")

f("random_drdi_activation_bop_%2A.csv",  "bop")

f("random_drdi_activation_as_%2A.csv",  "as")

+----------+------------+---------------+----------+----------------+--------------------+------------+----------+-------------+--------------------+
|COMMISSION|COUNT_CLIENT|DATE_CHARGEMENT| DATE_JOUR|DATE_TRANSACTION|               LOGIN|MOBILE_COACH| MOBILE_OM|NOMBRE_CLIENT|   NOM_PRENOMS_DU_MC|
+----------+------------+---------------+----------+----------------+--------------------+------------+----------+-------------+--------------------+
|    1500.0|        null|     2022-05-27|2022-04-24|            null| godogodo_iss_idt011|  0700274220|0747799758|            1|  KOUADIO YAO PRIVAT|
|    1500.0|        null|     2022-05-27|2022-05-11|            null|godogodo_iss_dist005|  0701566810|0708210020|            1|      OUATTARA SIAKA|
|    1500.0|        null|     2022-05-27|2022-04-27|            null|godogodo_iss_aman001|  0701566835|0709182047|            1|   OUATTARA SALY SIE|
|    1500.0|        null|     2022-05-27|2022-04-21|            null|   cover_iss_s2i_035|  07015667

In [ ]:
df_commission = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/youssouf/Downloads/COMMISSION_RECRUTEMENT_202203230.csv") \
      .dropDuplicates()


In [ ]:
df_commission = spark.read \
      .option("delimiter", ";") \
      .option("header", "true") \
      .csv("/Users/youssouf/Downloads/COMMISSION_RECRUTEMENT_202203230.csv") \
      .dropDuplicates()
